<a href="https://colab.research.google.com/github/harshhxrajputt/Pytorch/blob/main/Handwritten_digit_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# mnist is a dataset for digit
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform  = ToTensor(),
    download = True
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform  = ToTensor(),
    download = True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 39003398.13it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1183728.71it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9325799.13it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6537587.09it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
from torch.utils.data import DataLoader

Loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

}

In [7]:
Loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7b67e8e9b820>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7b67e8e9b4c0>}

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [15]:
class CNN( nn.Module ):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv1d(1,10,kernel_size=5)
    self.conv2 = nn.Conv2d(10,20,kernel_size=5)
    self.conv2.drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320,50)
    self.fc2 = nn.Linear(50,10)

  def forward(self,x):

      x = F.relu(F.max_pool2d(self.conv1(x),2) )
      x = F.relu(F.max_pool2d(self.conv2.drop(self.conv1(x)),2) )
      x = x.view(-1 , 320)
      x = F.relu(self.fc1(x))
      x = F.dropout(x , training = self.training)
      x = self.fc2(x)

      return F.softmax(x)    # softmax used for normalizing tensors to probability distributions




In [ ]:
import torch
